In [1]:
from IPython.core.display import display, HTML

def show(string):
    display(HTML(string))

# a. The Jinjia2 Template Engine

## 1.1 Rendering Templates

In [2]:
%%writefile templates/a_index.html
<h1>Hello World!<h1>

Overwriting templates/a_index.html


In [3]:
from flask import Flask, render_template


app = Flask(__name__)

@app.route('/')
def a_index():
    o = render_template('a_index.html'); show(o)
    return o

In [4]:
app_ctx = app.app_context()
app_ctx.push()

a_index()

'<h1>Hello World!<h1>'

In [5]:
%%writefile templates/a_user.html
<h1>Hello, {{name}}!<h1>

Overwriting templates/a_user.html


In [6]:
@app.route('/a_user/<name>')
def a_user(name):
    o = render_template('a_user.html', name = name); show(o)
    return o

In [7]:
a_user('Floyd')

'<h1>Hello, Floyd!<h1>'

## 1.2 Variables

In [8]:
%%writefile templates/a_variables.html
<p>A value from a dictionary: {{ mydict['key'] }}.</p>
<p>A value from a list: {{ mylist[3] }}.</p>
<p>A value from a list, with a variable index: {{ mylist[myintvar] }}.</p>
<p>A value from an object's method: {{ myobj.__len__() }}.</p>
<h3>Hello, {{name | capitalize}}!</h3>

Overwriting templates/a_variables.html


In [9]:
mydict ={}
mydict['key'] = 'a-value'
mylist = ['a', 'b', 'c', 'd']
myintvar = 3
myobj = mylist
myobj.__len__()

4

In [10]:
@app.route('/a_variables')
def a_variables(mydict, mylist, myintvar,myobj, name ):
    o = render_template('a_variables.html', 
                        mydict = mydict,
                        mylist = mylist,
                        myintvar=myintvar,
                        myobj  = myobj,
                        name = name); show(o)
    return o

In [11]:
app_ctx = app.app_context()
app_ctx.push()

a_variables(mydict, mylist, myintvar,myobj , name = 'floyd')

"<p>A value from a dictionary: a-value.</p>\n<p>A value from a list: d.</p>\n<p>A value from a list, with a variable index: d.</p>\n<p>A value from an object's method: 4.</p>\n<h3>Hello, Floyd!</h3>"

## 1.3 Control Structure

** 1. CONTROL **

In [12]:
%%writefile templates/a_control.html

{% if user %}
    <p>Hello, {{ user }}!</p>
{% else %}
    <p>Hello, Stranger!</p>
{% endif %}

<br>

<ul>
    {% for comment in comments %}
        <li>{{ comment }}</li>
    {% endfor %}
</ul>

<br>

<!- macro, just as define a function ->
{% macro render_comment(comment) %}
    <li>{{ comment }}</li>
{% endmacro %}

<ul>
    {% for comment in comments %}
        {{ render_comment(comment) }}
    {% endfor %}
</ul>

Overwriting templates/a_control.html


In [13]:
@app.route('/a_control')
def a_control(comments, user = None ):
    o = render_template('a_control.html', 
                         comments = comments,
                         user = user); show(o)
    return o

In [14]:
comments = ['comment1', 'jie2', 'jiejiejie3']
a_control(comments)

'\n\n    <p>Hello, Stranger!</p>\n\n\n<br>\n\n<ul>\n    \n        <li>comment1</li>\n    \n        <li>jie2</li>\n    \n        <li>jiejiejie3</li>\n    \n</ul>\n\n<br>\n\n<!- macro, just as define a function ->\n\n\n<ul>\n    \n        \n    <li>comment1</li>\n\n    \n        \n    <li>jie2</li>\n\n    \n        \n    <li>jiejiejie3</li>\n\n    \n</ul>'

** 2. MACRO **

In [15]:
%%writefile templates/a_macros.html

<!- macro, just as define a function ->
{% macro render_comment(comment) %}
    <li>{{ comment }}</li>
{% endmacro %}


Overwriting templates/a_macros.html


In [16]:
%%writefile templates/a_macros_import.html

{% import 'a_macros.html' as macros %}
<ul>
    {% for comment in comments %}
        {{ macros.render_comment(comment) }}
    {% endfor %}
</ul>


Overwriting templates/a_macros_import.html


In [17]:
@app.route('/a_macros_import')
def a_macros_import(comments):
    o = render_template('a_macros_import.html', 
                         comments = comments); show(o)
    return o

In [18]:
comments = ['comment1', 'jie2', 'jiejiejie3']
a_macros_import(comments)

'\n\n<ul>\n    \n        \n    <li>comment1</li>\n\n    \n        \n    <li>jie2</li>\n\n    \n        \n    <li>jiejiejie3</li>\n\n    \n</ul>'

** 3. INCLUDE common HTML **

In [19]:
%%writefile templates/a_common.html
<p>This is a common HTML, and we need to repeat it many times!</p>

Overwriting templates/a_common.html


In [20]:
%%writefile templates/a_common_include.html
<h3>First</h3>
{% include 'a_common.html' %}
<br>

<h3>Second</h3>
{% include 'a_common.html' %}
<br>

<h3>Third</h3>

{% include 'a_common.html' %}
<br>

Overwriting templates/a_common_include.html


In [21]:
@app.route('/a_common_include')
def a_common_include():
    o = render_template('a_common_include.html'); show(o)
    return o

In [22]:
a_common_include()

'<h3>First</h3>\n<p>This is a common HTML, and we need to repeat it many times!</p>\n<br>\n\n<h3>Second</h3>\n<p>This is a common HTML, and we need to repeat it many times!</p>\n<br>\n\n<h3>Third</h3>\n\n<p>This is a common HTML, and we need to repeat it many times!</p>\n<br>'

** 4. EXTEND base HTML **

In [23]:
%%writefile templates/a_base.html

<html>
<head>
    {% block head %}
    <title>{% block title %} {% endblock %} - My Application</title>
    {% endblock %}
</head>
<body>
    {% block body %}
    {% endblock %}
</body>
</html>

Overwriting templates/a_base.html


In [24]:
%%writefile templates/a_base_extend.html
{% extends "a_base.html" %}

<! -1- >
{% block title %}Index{% endblock %}

<! -2- >
{% block head %}
    {{ super() }}
    <style>
    </style>
{% endblock %}

<! -3- >
{% block body %}
<h1>Hello, World!</h1>
{% endblock %}

Overwriting templates/a_base_extend.html


In [25]:
@app.route('/a_base_extend')
def a_base_extend():
    o = render_template('a_base_extend.html'); show(o)
    return o

In [26]:
a_base_extend()

'\n<html>\n<head>\n    \n    \n    <title>Index - My Application</title>\n    \n    <style>\n    </style>\n\n</head>\n<body>\n    \n<h1>Hello, World!</h1>\n\n</body>\n</html>'

# b. Bootstrap Integration with Flask-Bootstrap

```shell
pip install flask-bootstrap
```

In [27]:
from IPython.core.display import display, HTML

def show(string):
    display(HTML(string))

In [28]:
from flask import Flask, render_template
from flask_bootstrap import Bootstrap

app = Flask(__name__)
bootstrap = Bootstrap(app)

In [29]:
%%writefile templates/b_user.html

{% extends "bootstrap/base.html" %}

{% block title %}Flasky{% endblock %}

{% block navbar %}
<div class="navbar navbar-inverse" role="navigation">
    <div class="container">
        <div class="navbar-header">
            <button type="button" class="navbar-toggle" data-toggle="collapse" data-target=".navbar-collapse">
                <span class="sr-only">Toggle navigation</span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
            </button>
            <a class="navbar-brand" href="/">Flasky</a>
        </div>
        <div class="navbar-collapse collapse">
            <ul class="nav navbar-nav">
                <li><a href="/">Home</a></li>
            </ul>
        </div>
    </div>
</div>
{% endblock %}

{% block content %}
<div class="container">
    <div class="page-header">
        <h1>Hello, {{ name }}!</h1>
    </div>
</div>
{% endblock %}

Overwriting templates/b_user.html


In [30]:
@app.route('/b_user/<name>')
def b_user(name):
    o = render_template('b_user.html', name=name); show(o)
    return o

In [31]:
app_ctx = app.app_context()
app_ctx.push()

In [32]:
b_user('Floyd')

'\n<!DOCTYPE html>\n<html>\n  <head>\n    <title>Flasky</title>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <!-- Bootstrap -->\n    <link href="//cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/3.3.7/css/bootstrap.min.css" rel="stylesheet">\n  </head>\n  <body>\n    \n<div class="navbar navbar-inverse" role="navigation">\n    <div class="container">\n        <div class="navbar-header">\n            <button type="button" class="navbar-toggle" data-toggle="collapse" data-target=".navbar-collapse">\n                <span class="sr-only">Toggle navigation</span>\n                <span class="icon-bar"></span>\n                <span class="icon-bar"></span>\n                <span class="icon-bar"></span>\n            </button>\n            <a class="navbar-brand" href="/">Flasky</a>\n        </div>\n        <div class="navbar-collapse collapse">\n            <ul class="nav navbar-nav">\n                <li><a href="/">Home</a></li>\n            </ul>\n 

# c. Custom Error Page

In [33]:
%%writefile templates/c_base.html
{% extends "bootstrap/base.html" %}

{% block title %}Flasky{% endblock %}

{% block navbar %}
<div class="navbar navbar-inverse" role="navigation">
    <div class="container">
        <div class="navbar-header">
            <button type="button" class="navbar-toggle"
             data-toggle="collapse" data-target=".navbar-collapse">
                <span class="sr-only">Toggle navigation</span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
            </button>
            <a class="navbar-brand" href="/">Flasky</a>
        </div>
        <div class="navbar-collapse collapse">
            <ul class="nav navbar-nav">
                <li><a href="/">Home</a></li>
            </ul>
        </div>
    </div>
</div>
{% endblock %}

{% block content %}
<div class="container">
    {% block page_content %}{% endblock %}
</div>
{% endblock %}

Overwriting templates/c_base.html


In [34]:
%%writefile templates/c_404.html
{% extends "c_base.html" %}

{% block title %}Flasky - Page Not Found{% endblock %}

{% block page_content %}
<div class="page-header">
    <h1>Not Found</h1>
</div>
{% endblock %}


Overwriting templates/c_404.html


In [35]:
%%writefile templates/c_500.html
{% extends "c_base.html" %}

{% block title %}Flasky - Internal Server Error{% endblock %}

{% block page_content %}
<div class="page-header">
    <h1>Internal Server Error</h1>
</div>
{% endblock %}


Overwriting templates/c_500.html


In [36]:
@app.errorhandler(404)
def page_not_found(e):
    o = render_template('c_404.html'); show(o)
    return o, 404

@app.errorhandler(500)
def internal_server_error(e):
    o = render_template('c_500.html'); show(o)
    return o, 500

In [37]:
page_not_found('e')

('<!DOCTYPE html>\n<html>\n  <head>\n    <title>Flasky - Page Not Found</title>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <!-- Bootstrap -->\n    <link href="//cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/3.3.7/css/bootstrap.min.css" rel="stylesheet">\n  </head>\n  <body>\n    \n<div class="navbar navbar-inverse" role="navigation">\n    <div class="container">\n        <div class="navbar-header">\n            <button type="button" class="navbar-toggle"\n             data-toggle="collapse" data-target=".navbar-collapse">\n                <span class="sr-only">Toggle navigation</span>\n                <span class="icon-bar"></span>\n                <span class="icon-bar"></span>\n                <span class="icon-bar"></span>\n            </button>\n            <a class="navbar-brand" href="/">Flasky</a>\n        </div>\n        <div class="navbar-collapse collapse">\n            <ul class="nav navbar-nav">\n                <li><a href="/">Home<

In [38]:
internal_server_error('e')

('<!DOCTYPE html>\n<html>\n  <head>\n    <title>Flasky - Internal Server Error</title>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <!-- Bootstrap -->\n    <link href="//cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/3.3.7/css/bootstrap.min.css" rel="stylesheet">\n  </head>\n  <body>\n    \n<div class="navbar navbar-inverse" role="navigation">\n    <div class="container">\n        <div class="navbar-header">\n            <button type="button" class="navbar-toggle"\n             data-toggle="collapse" data-target=".navbar-collapse">\n                <span class="sr-only">Toggle navigation</span>\n                <span class="icon-bar"></span>\n                <span class="icon-bar"></span>\n                <span class="icon-bar"></span>\n            </button>\n            <a class="navbar-brand" href="/">Flasky</a>\n        </div>\n        <div class="navbar-collapse collapse">\n            <ul class="nav navbar-nav">\n                <li><a href="/

# d. Links


`url_for`

In [39]:
app.config['SERVER_NAME'] = 'floydjjluo.me'

In [40]:
from flask import url_for


app_ctx = app.app_context()
app_ctx.push()


url_for('b_user', name = 'Floyd')

'http://floydjjluo.me/b_user/Floyd'

# e. Static Files

Get the icon.


![](static/favicon.ico)

In [41]:
url_for('static', filename='favicon.ico')

'http://floydjjluo.me/static/favicon.ico'

In [42]:
%%writefile templates/e_base.html
{% extends "bootstrap/base.html" %}

{% block title %}Flasky{% endblock %}

{% block head %}
{{ super() }}
<link rel="shortcut icon" href="{{ url_for('static', filename='favicon.ico') }}"
    type="image/x-icon">
<link rel="icon" href="{{ url_for('static', filename='favicon.ico') }}"
    type="image/x-icon">
{% endblock %} 


{% block navbar %}
<div class="navbar navbar-inverse" role="navigation">
    <div class="container">
        <div class="navbar-header">
            <button type="button" class="navbar-toggle"
             data-toggle="collapse" data-target=".navbar-collapse">
                <span class="sr-only">Toggle navigation</span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
            </button>
            <a class="navbar-brand" href="/">Flasky</a>
        </div>
        <div class="navbar-collapse collapse">
            <ul class="nav navbar-nav">
                <li><a href="/">Home</a></li>
            </ul>
        </div>
    </div>
</div>
{% endblock %}


{% block content %}
<div class="container">
    {% block page_content %}{% endblock %}
</div>
{% endblock %}

Overwriting templates/e_base.html


In [43]:
%%writefile templates/e_user.html

{% extends "e_base.html" %}

{% block title %}Flasky{% endblock %}

{% block page_content %}
<div class="page-header">
    <h1>Hello, {{ name }}!</h1>
</div>
{% endblock %}


Overwriting templates/e_user.html


In [44]:
@app.route('/e_user/<name>')
def e_user(name):
    o = render_template('e_user.html', name=name); show(o)
    return o

In [45]:
e_user('Floyd')

'\n<!DOCTYPE html>\n<html>\n  <head>\n\n    <title>Flasky</title>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <!-- Bootstrap -->\n    <link href="//cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/3.3.7/css/bootstrap.min.css" rel="stylesheet">\n<link rel="shortcut icon" href="http://floydjjluo.me/static/favicon.ico"\n    type="image/x-icon">\n<link rel="icon" href="http://floydjjluo.me/static/favicon.ico"\n    type="image/x-icon">\n\n  </head>\n  <body>\n    \n<div class="navbar navbar-inverse" role="navigation">\n    <div class="container">\n        <div class="navbar-header">\n            <button type="button" class="navbar-toggle"\n             data-toggle="collapse" data-target=".navbar-collapse">\n                <span class="sr-only">Toggle navigation</span>\n                <span class="icon-bar"></span>\n                <span class="icon-bar"></span>\n                <span class="icon-bar"></span>\n            </button>\n            <a class

In [46]:
app_ctx.pop()

# f. Localization of Dates and Times with Flask-Moment

In [1]:
from IPython.core.display import display, HTML

def show(string):
    display(HTML(string))

In [2]:
from flask import Flask, render_template
from flask_bootstrap import Bootstrap
from flask_moment import Moment

app = Flask(__name__)
bootstrap = Bootstrap(app)
moment = Moment(app)
moment

In [3]:
%%writefile templates/f_base.html
{% extends "bootstrap/base.html" %}

{% block title %}Flasky{% endblock %}

{% block head %}
{{ super() }}
<link rel="shortcut icon" href="{{ url_for('static', filename='favicon.ico') }}" type="image/x-icon">
<link rel="icon" href="{{ url_for('static', filename='favicon.ico') }}" type="image/x-icon">
{% endblock %}

{% block navbar %}
<div class="navbar navbar-inverse" role="navigation">
    <div class="container">
        <div class="navbar-header">
            <button type="button" class="navbar-toggle" data-toggle="collapse" data-target=".navbar-collapse">
                <span class="sr-only">Toggle navigation</span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
            </button>
            <a class="navbar-brand" href="/">Flasky</a>
        </div>
        <div class="navbar-collapse collapse">
            <ul class="nav navbar-nav">
                <li><a href="/">Home</a></li>
            </ul>
        </div>
    </div>
</div>
{% endblock %}

{% block content %}
<div class="container">
    {% block page_content %}{% endblock %}
</div>
{% endblock %}

<! -- New --- >
{% block scripts %}
{{ super() }}
{{ moment.include_moment() }}
{% endblock %}


Overwriting templates/f_base.html


In [4]:
from datetime import datetime

current_time = datetime.now()
current_time

datetime.datetime(2018, 5, 26, 13, 8, 29, 971614)

In [5]:
%%writefile templates/f_index.html

{% extends "f_base.html" %}

{% block title %}Flasky{% endblock %}

{% block page_content %}
<div class="page-header">
    <h1>Hello World!</h1>
</div>

<p>The local date and time is {{ moment(current_time).format('LLL') }}.</p>

<p>That was {{ moment(current_time).fromNow(refresh=True) }}.</p>
{% endblock %}


Overwriting templates/f_index.html


In [6]:
@app.route('/f_index/')
def f_index():
    o = render_template('f_index.html', current_time=datetime.utcnow()); show(o)
    return o

In [8]:
app.config['SERVER_NAME'] = 'floydjjluo.me'
app_ctx = app.app_context()
app_ctx.push()

f_index()

'\n<!DOCTYPE html>\n<html>\n  <head>\n\n    <title>Flasky</title>\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <!-- Bootstrap -->\n    <link href="//cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/3.3.7/css/bootstrap.min.css" rel="stylesheet">\n<link rel="shortcut icon" href="http://floydjjluo.me/static/favicon.ico" type="image/x-icon">\n<link rel="icon" href="http://floydjjluo.me/static/favicon.ico" type="image/x-icon">\n\n  </head>\n  <body>\n    \n<div class="navbar navbar-inverse" role="navigation">\n    <div class="container">\n        <div class="navbar-header">\n            <button type="button" class="navbar-toggle" data-toggle="collapse" data-target=".navbar-collapse">\n                <span class="sr-only">Toggle navigation</span>\n                <span class="icon-bar"></span>\n                <span class="icon-bar"></span>\n                <span class="icon-bar"></span>\n            </button>\n            <a class="navbar-brand" href="/"